# Evaluation - stage cartographie de l'écosystème

Ce notebook est à réaliser chez soi dans un délai de 2 jours, mais rassurez-vous, ça ne prend pas autant de temps ! Il a pour objectif d'évaluer certaines de vos capacités en code et vos bonnes pratiques en scraping, NLP (Natural Language Processing) et manipulation d'API.

L'objectif de ce notebook n'est pas de produire des outils opérationnels, mais d'évaluer vos compétences ou votre capacité à chercher des informations.

En cas de doute ou de question, n'hésitez pas à nous écrire.

# 0. Git

La première chose à faire est de "récupérer" ce notebook pour le compléter. Plusieurs options selon votre maîtrise de Git et/ou que vous ayez un compte :
- fork le repo suivant et le compléter sur un projet perso
- vous ajouter au projet, créez votre branch `submission_nomprenom` et push dessus
- vous envoyez par mail le notebook à compléter et renvoyer par mail

https://github.com/TitouanBlaize/carto_affi/blob/main/evaluation_git.ipynb

## 1. NLP

Cette première partie évalue vos connaissance des packages de NLP et vos bonnes pratiques quant au preprocessing de texte. 

In [ ]:
# écrivez ici une liste de librairies que vous connaissez pour faire du NLP
import spacy
from spacy import displacy 
import fr_core_news_md

Dans une première sous-partie, nous vous proposons de coder un pipeline/fonction de preprocessing de données textuelles. 

L'objectif est, à partir de textes bruts, de produire des textes propres (ponctuation, casse, pluriel, enlever les stopwords,...) qui peuvent être utilisés en entrée d'un pipeline d'apprentissage/fonction. 
Libre à vous d'inclure et d'implémenter toutes les fonctions de nettoyage que vous souhaitez.
Ces opérations peuvent être menées au sein d'une classe (cf class NLPPipeline) ou d'une fonction (NLP_cleaning), comme vous préférez.

In [ ]:
# Via une classe ("Orienté objet")
class NLPPipeline():
    def __init__(self):
      nlp = fr_core_news_md.load()
      pass
    
    def transform(self, x):
      for doc in nlp.pipe([x], disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"]):
          print([(e.text, e.label_) for e in doc.ents])
      pass

In [ ]:
pipeline = NLPPipeline()
pipeline.transform("Georges Washington, 1er président des USA, habite à Washington.")


[('Georges Washington', 'PERSON'), ('1er', 'ORDINAL'), ('USA', 'GPE'), ('Washington', 'GPE')]


In [ ]:
# Via une fonction
import string
import re
import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

def NLP_cleaning(x):
    final_str = ""
    # lowercase
    text = x.lower()
    # supprimer le retour à la ligne
    text = re.sub(r'\n', '', text)
    # Supprime la ponctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    # Supprimer les stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("french")
    text_filtered = [word for word in text if not word in useless_words]
    # Supprimer les chiffres
    text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]

    final_str = ' '.join(text_filtered)
    return final_str

In [ ]:
NLP_cleaning("Georges Washington, 1er président des USA, habite à Washington.")

'georges washington  président usa habite washington'

On pourra par exemple essayer d'obtenir `georges washington er president usa habite a washington`, il n'y a pas de "bon" résultat, libre à vous d'ajouter plusieurs cleaning de textes.

## 2. Scraping/API

Dans cette deuxième partie, nous vous proposons d'écrire une petite fonction de scraping.

A l'aide de la librairie de votre choix, écrivez une fonction qui permet de trouver le nombre de citations d'un article sur Pubmed.   
Vous appliquerez la fonction à l'article suivant: "New onset refractory status epilepticus (NORSE) *C. Sculier, N. Gaspard*, 2019" dont l'ID Pubmed est 30482654

https://pubmed.ncbi.nlm.nih.gov/30482654/

In [ ]:
from bs4 import BeautifulSoup 
import requests 

In [ ]:
def get_number_of_citations(pubmed_id):
    """
    get the number of citations of a given pubmed article
    """
    r = requests.get('https://pubmed.ncbi.nlm.nih.gov/?linkname=pubmed_pubmed_citedin&from_uid=' + str(pubmed_id)) 
    soup = BeautifulSoup(r.text, 'html.parser')
    all_results_amount = soup.find_all('div', attrs={'class': 'results-amount'})
    #Test sur si l'aricle est cité ou pas
    if all_results_amount:
      print(int(all_results_amount[0].find_all('span', attrs={'class': 'value'})[0].text))
    else:
      print(0)
    pass

En runnant la case suivante on est censé obtenir 19, bonne chance !

In [ ]:
get_number_of_citations(30482654)

19


In [ ]:
#cas d'aricle non cité
get_number_of_citations(30482653)

0
